In [1]:

import pandas as pd
import numpy as np
import os 
import gzip
import json
import pickle
import pandas as pd  # type: ignore
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    precision_score, recall_score, f1_score, balanced_accuracy_score,
    confusion_matrix
)

In [2]:
import os
print("Working directory:", os.getcwd())

Working directory: c:\Users\valen\Documents\Maestria\Analitica Predictiva\LAB-01-prediccion-del-default-usando-rf-vatamayog\homework


In [ ]:
# os.chdir("..")
# os.getcwd()

'c:\\Users\\valen\\Documents\\Maestria\\Analitica Predictiva\\LAB-01-prediccion-del-default-usando-rf-vatamayog'

 # Paso 1.
 Realice la limpieza de los datasets:
 - Renombre la columna "default payment next month" a "default".
 - Remueva la columna "ID".
 - Elimine los registros con informacion no disponible.
 - Para la columna EDUCATION, valores > 4 indican niveles superiores

In [4]:
# ============================
# Paso 1. Cargar y limpiar datos
# ============================


def load_data():
    train_path = 'files/input/train_data.csv.zip'
    test_path = 'files/input/test_data.csv.zip'

    train_dataset = pd.read_csv(train_path, index_col=False)
    test_dataset = pd.read_csv(test_path, index_col=False)
    return train_dataset, test_dataset



In [5]:
train_dataset, test_dataset = load_data()

In [6]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          21000 non-null  int64
 1   LIMIT_BAL                   21000 non-null  int64
 2   SEX                         21000 non-null  int64
 3   EDUCATION                   21000 non-null  int64
 4   MARRIAGE                    21000 non-null  int64
 5   AGE                         21000 non-null  int64
 6   PAY_0                       21000 non-null  int64
 7   PAY_2                       21000 non-null  int64
 8   PAY_3                       21000 non-null  int64
 9   PAY_4                       21000 non-null  int64
 10  PAY_5                       21000 non-null  int64
 11  PAY_6                       21000 non-null  int64
 12  BILL_AMT1                   21000 non-null  int64
 13  BILL_AMT2                   21000 non-null  int64
 14  BILL_A

In [7]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          9000 non-null   int64
 1   LIMIT_BAL                   9000 non-null   int64
 2   SEX                         9000 non-null   int64
 3   EDUCATION                   9000 non-null   int64
 4   MARRIAGE                    9000 non-null   int64
 5   AGE                         9000 non-null   int64
 6   PAY_0                       9000 non-null   int64
 7   PAY_2                       9000 non-null   int64
 8   PAY_3                       9000 non-null   int64
 9   PAY_4                       9000 non-null   int64
 10  PAY_5                       9000 non-null   int64
 11  PAY_6                       9000 non-null   int64
 12  BILL_AMT1                   9000 non-null   int64
 13  BILL_AMT2                   9000 non-null   int64
 14  BILL_AMT

In [8]:
# Verificar si los tipos de datos son los mismos en train y test
def validate_data_types(train_df, test_df):
    train_dtypes = train_df.dtypes
    test_dtypes = test_df.dtypes

    # Comparar los tipos de datos
    mismatched_columns = [
        column for column in train_dtypes.index
        if column in test_dtypes.index and train_dtypes[column] != test_dtypes[column]
    ]

    if mismatched_columns:
        print("Las siguientes columnas tienen tipos de datos diferentes entre train y test:")
        for column in mismatched_columns:
            print(f"Columna: {column}, Train: {train_dtypes[column]}, Test: {test_dtypes[column]}")
    else:
        print("Los tipos de datos son consistentes entre train y test.")

# Llamar a la función con los DataFrames de entrenamiento y prueba
validate_data_types(train_dataset, test_dataset)

Los tipos de datos son consistentes entre train y test.


In [9]:
train_dataset.shape, test_dataset.shape

((21000, 25), (9000, 25))

In [10]:
train_dataset.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,10748,310000,1,3,1,32,0,0,0,0,...,84373,57779,14163,8295,6000,4000,3000,1000,2000,0
1,12574,10000,2,3,1,49,-1,-1,-2,-1,...,1690,1138,930,0,0,2828,0,182,0,1
2,29677,50000,1,2,1,28,-1,-1,-1,0,...,45975,1300,43987,0,46257,2200,1300,43987,1386,0
3,8857,80000,2,3,1,52,2,2,3,3,...,40748,39816,40607,3700,1600,1600,0,1600,1600,1
4,21099,270000,1,1,2,34,1,2,0,0,...,22448,15490,17343,0,4000,2000,0,2000,2000,0


In [11]:
test_dataset.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
1,10,20000,1,3,2,35,-2,-2,-2,-2,...,0,13007,13912,0,0,0,13007,1122,0,0
2,11,200000,2,3,2,34,0,0,2,0,...,2513,1828,3731,2306,12,50,300,3738,66,0
3,15,250000,1,1,2,29,0,0,0,0,...,59696,56875,55512,3000,3000,3000,3000,3000,3000,0
4,16,50000,2,3,3,23,1,2,0,0,...,28771,29531,30211,0,1500,1100,1200,1300,1100,0


In [12]:
import pandas as pd
import zipfile
import numpy as np


def clean_dataset(df: pd.DataFrame) -> pd.DataFrame:
    df.rename(columns={"default payment next month": "default"}, inplace=True)

    df.drop(columns=["ID"], inplace=True)

    df = df.loc[df["MARRIAGE"] != 0]
    df = df.loc[df["EDUCATION"] != 0]

    df["EDUCATION"] = df["EDUCATION"].apply(lambda x: 4 if x > 4 else x)

    df.dropna(inplace=True)

    # Eliminar duplicados
    df = df.drop_duplicates()

    return df


# Limpiar datasets

train_dataset = clean_dataset(train_dataset)
test_dataset = clean_dataset(test_dataset)

In [13]:
print("Valores únicos en y_train:", train_dataset["default"].unique())
print("Valores únicos en y_test:", test_dataset["default"].unique())
print("tipo de datos y_train:", train_dataset["default"].dtype)
print("tipo de datos y_test:", test_dataset["default"].dtype)

Valores únicos en y_train: [0 1]
Valores únicos en y_test: [1 0]
tipo de datos y_train: int64
tipo de datos y_test: int64


# Paso 2.
Divida los datasets en x_train, y_train, x_test, y_test.

In [14]:
def split_features_target(train_dataset, test_dataset, target_col="default"):
    x_train = train_dataset.drop(columns=[target_col])
    y_train = train_dataset[target_col].astype(int).values.ravel()
    x_test = test_dataset.drop(columns=[target_col])
    y_test = test_dataset[target_col].astype(int).values.ravel()
    return x_train, y_train, x_test, y_test

In [15]:
x_train, y_train, x_test, y_test = split_features_target(train_dataset, test_dataset, "default")


print("Tamaños:")
print("x_train:", x_train.shape)
print("y_train:", y_train.shape)
print("x_test:", x_test.shape)
print("y_test:", y_test.shape)

Tamaños:
x_train: (20934, 23)
y_train: (20934,)
x_test: (8974, 23)
y_test: (8974,)


# Paso 3.
 Cree un pipeline para el modelo de clasificación. Este pipeline debe
 contener las siguientes capas:
 - Transforma las variables categoricas usando el método
   one-hot-encoding.
 - Ajusta un modelo de bosques aleatorios (rando forest).

In [16]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20934 entries, 0 to 20999
Data columns (total 23 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   LIMIT_BAL  20934 non-null  int64
 1   SEX        20934 non-null  int64
 2   EDUCATION  20934 non-null  int64
 3   MARRIAGE   20934 non-null  int64
 4   AGE        20934 non-null  int64
 5   PAY_0      20934 non-null  int64
 6   PAY_2      20934 non-null  int64
 7   PAY_3      20934 non-null  int64
 8   PAY_4      20934 non-null  int64
 9   PAY_5      20934 non-null  int64
 10  PAY_6      20934 non-null  int64
 11  BILL_AMT1  20934 non-null  int64
 12  BILL_AMT2  20934 non-null  int64
 13  BILL_AMT3  20934 non-null  int64
 14  BILL_AMT4  20934 non-null  int64
 15  BILL_AMT5  20934 non-null  int64
 16  BILL_AMT6  20934 non-null  int64
 17  PAY_AMT1   20934 non-null  int64
 18  PAY_AMT2   20934 non-null  int64
 19  PAY_AMT3   20934 non-null  int64
 20  PAY_AMT4   20934 non-null  int64
 21  PAY_AMT5   20934 

In [17]:
def make_pipeline(list_categorical, estimator):

    preprocessor = ColumnTransformer(
        transformers=[
            ("onehot", OneHotEncoder(handle_unknown="ignore"), list_categorical)
        ],
        remainder="passthrough"
    )

    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", estimator)
        ]
    )

    return pipeline

In [18]:
list_categorical = ["EDUCATION", "MARRIAGE", "SEX"]

pipeline = make_pipeline(list_categorical, RandomForestClassifier(random_state=42))

#  Paso 4.
- Optimice los hiperparametros del pipeline usando validación cruzada.
- se 10 splits para la validación cruzada. Use la función de precision
- balanceada para medir la precisión del modelo.


In [19]:
def make_grid_search(pipeline, param_grid, cv, score, x_train, y_train):

    from sklearn.model_selection import GridSearchCV

    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=cv,
        scoring=score,
        n_jobs=-1,
        verbose=2
    )

    grid_search.fit(x_train, np.ravel(y_train))

    return grid_search


# Paso 5.
- Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
- Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

In [20]:
def save_estimator(estimator):
    models_path = "files/models"
    os.makedirs(models_path, exist_ok=True)

    with gzip.open("files/models/model.pkl.gz", "wb") as file:
        pickle.dump(estimator, file)     
    print(f"Modelo guardado en: {'files/models/model.pkl.gz'}")


def load_estimator(output_path):
    """Cargar modelo comprimido"""
    import gzip, pickle
    if not os.path.exists(output_path):
        return None
    with gzip.open(output_path, "rb") as f:
        return pickle.load(f)



In [ ]:
param_grid = {
   "classifier__n_estimators": [30, 50, 100],
   "classifier__max_depth": [None, 10, 20],
   "classifier__min_samples_split": [5, 10],
   "classifier__min_samples_leaf": [2, 4],
   "classifier__max_features": [10, 25]
}

In [22]:
model = make_grid_search(pipeline, param_grid, cv=10, score="balanced_accuracy", x_train=x_train, y_train=y_train)


Fitting 10 folds for each of 1 candidates, totalling 10 fits


In [23]:
# guardar el modelo entrenado
save_estimator(model)

Modelo guardado en: files/models/model.pkl.gz


In [24]:
current_components = [str(model.estimator[i]) for i in range(len(model.estimator))]
current_components

["ColumnTransformer(remainder='passthrough',\n                  transformers=[('onehot',\n                                 OneHotEncoder(handle_unknown='ignore'),\n                                 ['EDUCATION', 'MARRIAGE', 'SEX'])])",
 'RandomForestClassifier(random_state=42)']

# Paso 6.
 Calcule las metricas de precision, precision balanceada, recall,
 y f1-score para los conjuntos de entrenamiento y prueba.
 Guardelas en el archivo files/output/metrics.json. Cada fila
 del archivo es un diccionario con las metricas de un modelo.
 Este diccionario tiene un campo para indicar si es el conjunto
 de entrenamiento o prueba. Por ejemplo:

 {'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
 {'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}

# Paso 7.
 Calcule las matrices de confusion para los conjuntos de entrenamiento y
 prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
 del archivo es un diccionario con las metricas de un modelo.
 de entrenamiento o prueba. Por ejemplo:

 {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
 {'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}


In [25]:
def calculate_metrics(model, x_train, y_train, x_test, y_test):

    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    cm_train = confusion_matrix(y_train, y_train_pred)
    cm_test = confusion_matrix(y_test, y_test_pred)

    metrics = [
        {
            'type': 'metrics',
            'dataset': 'train',
            'precision': precision_score(y_train, y_train_pred, zero_division=0),
            'balanced_accuracy': balanced_accuracy_score(y_train, y_train_pred),
            'recall': recall_score(y_train, y_train_pred, zero_division=0),
            'f1_score': f1_score(y_train, y_train_pred, zero_division=0)
        },
        {
            'type': 'metrics',
            'dataset': 'test',
            'precision': precision_score(y_test, y_test_pred, zero_division=0),
            'balanced_accuracy': balanced_accuracy_score(y_test, y_test_pred),
            'recall': recall_score(y_test, y_test_pred, zero_division=0),
            'f1_score': f1_score(y_test, y_test_pred, zero_division=0)
        },
        {
            'type': 'cm_matrix',
            'dataset': 'train',
            'true_0': {'predicted_0': int(cm_train[0, 0]), 'predicted_1': int(cm_train[0, 1])},
            'true_1': {'predicted_0': int(cm_train[1, 0]), 'predicted_1': int(cm_train[1, 1])}
        },
        {
            'type': 'cm_matrix',
            'dataset': 'test',
            'true_0': {'predicted_0': int(cm_test[0, 0]), 'predicted_1': int(cm_test[0, 1])},
            'true_1': {'predicted_0': int(cm_test[1, 0]), 'predicted_1': int(cm_test[1, 1])}
        }
    ]

    return metrics

def save_metrics(metrics):
    metrics_path = "files/output"
    os.makedirs(metrics_path, exist_ok=True)
    
    with open("files/output/metrics.json", "w") as file:
        for metric in metrics:
            file.write(json.dumps(metric, ensure_ascii=False))
            file.write('\n')

    with open("files/output/metrics.json", "r") as file:
        print("Contenido de metrics.json:")
        print(file.read())

In [26]:
metrics = calculate_metrics(model, x_train, y_train, x_test, y_test)

save_metrics(metrics)

Contenido de metrics.json:
{"type": "metrics", "dataset": "train", "precision": 0.9466415568110483, "balanced_accuracy": 0.8139773224105086, "recall": 0.6384419983065199, "f1_score": 0.7625790139064476}
{"type": "metrics", "dataset": "test", "precision": 0.6578947368421053, "balanced_accuracy": 0.6750182003494467, "recall": 0.4070378151260504, "f1_score": 0.5029201817001947}
{"type": "cm_matrix", "dataset": "train", "true_0": {"predicted_0": 16040, "predicted_1": 170}, "true_1": {"predicted_0": 1708, "predicted_1": 3016}}
{"type": "cm_matrix", "dataset": "test", "true_0": {"predicted_0": 6667, "predicted_1": 403}, "true_1": {"predicted_0": 1129, "predicted_1": 775}}



## TEST

In [27]:
## primera parte del test

MODEL_FILENAME = "files/models/model.pkl.gz"
MODEL_COMPONENTS = [
    "OneHotEncoder",
    "RandomForestClassifier",
]
SCORES = [
    0.785,
    0.673,
]


# ------------------------------------------------------------------------------
#
# Internal tests
#
def _load_model():
    """Generic test to load a model"""
    assert os.path.exists(MODEL_FILENAME)
    with gzip.open(MODEL_FILENAME, "rb") as file:
        model = pickle.load(file)
    assert model is not None
    return model

## parte 1

In [28]:
model = _load_model()

## parte 2

In [29]:
def _load_grading_data():
    """Load grading data"""
    with open("files/grading/x_train.pkl", "rb") as file:
        x_train = pickle.load(file)

    with open("files/grading/y_train.pkl", "rb") as file:
        y_train = pickle.load(file)

    with open("files/grading/x_test.pkl", "rb") as file:
        x_test = pickle.load(file)

    with open("files/grading/y_test.pkl", "rb") as file:
        y_test = pickle.load(file)

    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = _load_grading_data()

## parte 3

In [30]:
def _load_metrics():
    assert os.path.exists("files/output/metrics.json")
    metrics = []
    with open("files/output/metrics.json", "r", encoding="utf-8") as file:
        for line in file:
            metrics.append(json.loads(line))
    return metrics

In [31]:
metrics = _load_metrics()

## parte 4 

In [32]:
def _test_components(model):
    """Test components"""
    assert "GridSearchCV" in str(type(model))
    current_components = [str(model.estimator[i]) for i in range(len(model.estimator))]
    for component in MODEL_COMPONENTS:
        assert any(component in x for x in current_components)

In [33]:
_test_components(model)

In [34]:
[str(model.estimator[i]) for i in range(len(model.estimator))]

["ColumnTransformer(remainder='passthrough',\n                  transformers=[('onehot',\n                                 OneHotEncoder(handle_unknown='ignore'),\n                                 ['EDUCATION', 'MARRIAGE', 'SEX'])])",
 'RandomForestClassifier(random_state=42)']

## parte 5

In [35]:
SCORES = [
    0.785,
    0.673,
]
def _test_scores(model, x_train, y_train, x_test, y_test):
    """Test scores"""
    assert model.score(x_train, y_train) > SCORES[0]
    assert model.score(x_test, y_test) > SCORES[1]

In [36]:
def _load_grading_data():
    """Load grading data"""
    with open("files/grading/x_train.pkl", "rb") as file:
        x_train = pickle.load(file)

    with open("files/grading/y_train.pkl", "rb") as file:
        y_train = pickle.load(file)

    with open("files/grading/x_test.pkl", "rb") as file:
        x_test = pickle.load(file)

    with open("files/grading/y_test.pkl", "rb") as file:
        y_test = pickle.load(file)

    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = _load_grading_data()

In [37]:
def _load_model():
    with gzip.open("files/models/model.pkl.gz", "rb") as file:
        model = pickle.load(file)
    return model

In [38]:
model = _load_model()

In [39]:
_test_scores(model, x_train, y_train, x_test, y_test)

In [40]:
model.score(x_train, y_train)

0.8140213987712139

In [41]:
model.score(x_test, y_test)

0.6750790623165492

In [42]:
model.score(x_train, y_train) > SCORES[0]

True

In [43]:
model.score(x_test, y_test) > SCORES[1]

True

In [44]:
!where python

c:\Users\valen\Documents\Maestria\Analitica Predictiva\LAB-01-prediccion-del-default-usando-rf-vatamayog\.venv\Scripts\python.exe
C:\Users\valen\AppData\Local\Programs\Python\Python311\python.exe
C:\Users\valen\AppData\Local\Microsoft\WindowsApps\python.exe


In [49]:
!pytest -v

============================= test session starts =============================
platform win32 -- Python 3.11.8, pytest-8.4.2, pluggy-1.6.0 -- C:\Users\valen\Documents\Maestria\Analitica Predictiva\LAB-01-prediccion-del-default-usando-rf-vatamayog\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\valen\Documents\Maestria\Analitica Predictiva\LAB-01-prediccion-del-default-usando-rf-vatamayog
collecting ... collected 1 item

tests/test_homework.py::test_homework PASSED                             [100%]

============================== 1 passed in 4.02s ==============================
